# José Ligorría
## Deployment modelos predicción sobrevivencia de pasajeros en el Titanic

In [115]:
import glob
from sklearn.externals import joblib
import pandas as pd
from sklearn import tree
from sklearn import svm
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from collections import Counter
import numpy as np
import tensorflow as tf

def ClasificadorBinario(pred_x,pendientes,interceptos,scale,mean,sd):
    if scale:
        pred_x = (pred_x-mean.astype(np.float32))/sd.astype(np.float32)
        pred_f = tf.argmax(tf.add(tf.tensordot(pred_x, pendientes,axes = [[1], [0]]), interceptos),1)
    else:
        pred_f = tf.argmax(tf.add(tf.tensordot(pred_x.astype(np.float32), pendientes,axes = [[1], [0]]), interceptos),1)
    with tf.Session() as sess: 
        pred_f_val = pred_f.eval()
    return pred_f_val

In [116]:
if tf.__version__.startswith("2."):
    import tensorflow.compat.v1 as tf
    tf.compat.v1.disable_v2_behavior()
    tf.compat.v1.disable_eager_execution()
    print("Enabled compatitility to tf1.x")

Instructions for updating:
non-resource variables are not supported in the long term
Enabled compatitility to tf1.x


In [128]:
input_dir = "./Files/"  + 'Diccionario_Variables' + '.pkl'

diccionarioVariables =  joblib.load(input_dir)

Best_Dec = glob.glob('./Objects/Best*Dec*.pkl')
best_model_dt =  joblib.load(Best_Dec[0])
Best_SVM = glob.glob('./Objects/Best*SV*.pkl')
best_model_sv =  joblib.load(Best_SVM[0])
Best_Nne = glob.glob('./Objects/Best*Nn*.pkl')
best_model_nn =  joblib.load(Best_Nne[0])
Best_Reg = glob.glob('./Objects/Best*Reg*.pkl')
best_model_lr =  joblib.load(Best_Reg[0])

Best_Dec_split = Best_Dec[0].split('_')
vars_necesitadas_Dec = [x for x in Best_Dec_split if 'var' in x]
Best_SVM_split = Best_SVM[0].split('_')
vars_necesitadas_SVM = [x for x in Best_SVM_split if 'var' in x]
Best_Nne_split = Best_Nne[0].split('_')
vars_necesitadas_Nne = [x for x in Best_Nne_split if 'var' in x]
Best_Reg_split = Best_Reg[0].split('_')
vars_necesitadas_Reg = [x for x in Best_Reg_split if 'var' in x]
vars_necesarias = list(set(vars_necesitadas_Dec) | set(vars_necesitadas_SVM) | set(vars_necesitadas_Nne) | set(vars_necesitadas_Reg))
variables_necesarias = [diccionarioVariables[x] for x in vars_necesarias]
#print('Las variables necesarias para generar una prediccion son:', string_variables)

def Prediccion(X):
    arr_X = X.reshape((1,-1))
    pred_ct_test = best_model_dt.predict(arr_X)
    pred_sv_test = best_model_sv.predict(arr_X)
    pred_rl_test = ClasificadorBinario(arr_X,best_model_lr[0],best_model_lr[1],best_model_lr[2],best_model_lr[3],best_model_lr[4])
    pred_nn_test = best_model_nn.predict(arr_X)
    pred_em_test = pred_ct_test + pred_sv_test + pred_rl_test + pred_nn_test
    pred_em_test[pred_em_test<=2]=0
    pred_em_test[pred_em_test>=2]=1
    return int(pred_em_test)

def EvalPrediccion(X):
    try:
        df_X = pd.DataFrame(data=X, index=[0])
        variables_en_X = list(df_X.columns)
        print(variables_en_X)
        if len(variables_en_X) == len(vars_necesarias):
            set_difference = set(variables_necesarias) - set(variables_en_X)
            #print(set_difference)
            if set_difference == []:
                #print('Probable')
                return Prediccion(df_X)
            else:
                print('Corregir el nombre de las variables y probar de nuevo')
        else:
            print('Corregir la cantidad de variables y probar de nuevo')
    except:
        arr_X = np.array(X,dtype = np.float32)
        if len(arr_X) == len(vars_necesarias):
            #print('Probable')
            return Prediccion(arr_X)


In [133]:
df = pd.read_csv ('data_titanic_proyecto.csv')
dataset = df[['SibSp','Parch','Fare','Embarked','passenger_class','passenger_sex','passenger_survived']]
reg_out = [False]*len(dataset)
for f in ['SibSp', 'Parch','Fare','Embarked','passenger_class','passenger_sex']:
    piv_f = dataset[f]
    #print(f,len(Counter(piv_f)),len(piv_f[piv_f.isnull()]))
    reg_out += piv_f.isnull()
dataset = dataset[-reg_out]
dataset = dataset.reset_index()
for f in ['Embarked','passenger_class','passenger_sex']:
    labels = dataset[f]
    distinct_labels = list(set(labels))
    labels_num = np.zeros_like(labels)
    contador = 0
    total = 0
    total_ant = 0
    cantidad = 0
    for i in distinct_labels:
        contador += 1
        piv1 = labels == i
        labels_num[piv1] = contador
        total = sum(labels_num)
        cantidad += (total-total_ant)/contador
        total_ant = total
    #print(sum(labels_num),cantidad)
    labels_num -= 1
    labels_num = labels_num.astype(int)
    labels_num_eye = np.eye(len(set(labels_num)))[labels_num]
    labels_num_eye =  pd.DataFrame(data=labels_num_eye,columns=[f +'_' + s for s in distinct_labels]  )
    len(labels_num_eye)
    dataset = dataset.join(labels_num_eye)
    dataset.pop(f)
labels = dataset['passenger_survived']
labels_num = np.zeros_like(labels)
labels_num[labels =='Y'] = 1
dataset['passenger_survived'] = labels_num
dataset = dataset[['SibSp','Parch','Fare','Embarked_S','Embarked_Q','Embarked_C','passenger_class_Lower',
                  'passenger_class_Middle','passenger_class_Upper','passenger_sex_M','passenger_sex_F',
                  'passenger_survived']]
dataset = dataset.sample(n=10)
X_test = np.array(dataset[['SibSp','Parch','Fare','Embarked_S','Embarked_Q','Embarked_C','passenger_class_Lower',
                      'passenger_class_Middle','passenger_class_Upper','passenger_sex_M','passenger_sex_F']],dtype = np.float32)
y_test = np.array(dataset[['passenger_survived']],dtype = np.float32)
for i in range(len(dataset)):
    dataset_piv = X_test[i]
    #print(i)
    print('Prediccion: ',EvalPrediccion(dataset_piv),' real:', y_test[i])
#len(dataset)
#pd.DataFrame(data=dataset)

C:\Users\jose-\Anaconda3\envs\Clases_tensorflow_env2\lib\site-packages\pandas\core\computation\expressions.py:178: UserWarning: evaluating in Python space because the '+' operator is not supported by numexpr for the bool dtype, use '|' instead
  f"evaluating in Python space because the {repr(op_str)} "


Prediccion:  1  real: [1.]
Prediccion:  0  real: [1.]
Prediccion:  1  real: [0.]
Prediccion:  1  real: [1.]
Prediccion:  0  real: [0.]
Prediccion:  0  real: [1.]
Prediccion:  1  real: [1.]
Prediccion:  0  real: [0.]
Prediccion:  0  real: [1.]
Prediccion:  0  real: [0.]
